In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [4]:
video_files = glob.glob('video_footages/*')

In [5]:
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\vecteezy_busy-street-in-evening-hong-kong_28840705 (1) (1).mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4']

In [253]:
cap = cv.VideoCapture(video_files[0])

first_frame = None

kernel = np.array([[0, -2, 0], [-2, 10, -2], [0, -2, 0]])

kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 15))


while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    # Display the resulting fram
    height, width, layers = frame.shape
    new_h = height / 2
    new_w = width / 2
    frame = cv.resize(frame, (int(new_w), int(new_h)))
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (15,15), 0)
    if first_frame is None:
        first_frame = gray
        continue
    
    delta_frame = cv.absdiff(first_frame, gray)

    #thresholding:
    delta_frame = cv.threshold(delta_frame, 30, 255, cv.THRESH_BINARY)[1]
    

    #sharpening:
    #delta_frame = cv.filter2D(delta_frame, -1, kernel)

    
    
    #dilation:
    delta_frame = cv.dilate(delta_frame, None, iterations=2)

    #morphing:
    #delta_frame = cv.morphologyEx(delta_frame, cv.MORPH_DILATE, kernel_morph)


    contours, hiarchy = cv.findContours(delta_frame, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    #cv.drawContours(delta_frame, contours, -1, (255,255,255), cv.FILLED)

    for cnt in contours:
        if cv.contourArea(cnt)<300:
            continue
        (x,y,w,h) = cv.boundingRect(cnt)
        cv.rectangle(frame,(x,y), (x+w,y+h), (0,255,0), 2)

    
            
    cv.imshow('normal video',frame)
    cv.imshow('difference video', delta_frame)

    
 
    # Press Q on keyboard to  exit
    if cv.waitKey(30) & 0xFF == ord('q'):
      break

    first_frame = gray
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv.destroyAllWindows()

In [217]:
image_files = glob.glob('images/*')

In [110]:
image_files

['images\\test.png',
 'images\\woman1.jpg',
 'images\\woman2.jpg',
 'images\\woman3.jpg']

In [148]:
img = cv.imread(image_files[0], cv.IMREAD_GRAYSCALE)

height, width= img.shape
new_h = height / 2
new_w = width / 2
img = cv.resize(img, (int(new_w), int(new_h)))

thresh = cv.threshold(img, 128, 255, cv.THRESH_BINARY)[1]

contours, hierchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

cv.drawContours(thresh, contours, -1, (255,255,255), cv.FILLED)

# save results
cv.imshow('original_image', img)
cv.imshow('result', thresh)
cv.waitKey(0)
cv.destroyAllWindows()